In [1]:
%config IPCompleter.greedy = True
%autosave 60

Autosaving every 60 seconds


## 1. NLP

### Cargar archivo

In [14]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2').sample(2000, replace=False)
# df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2')
df_criticas

,Review,Label
45748,It's really unfortunate that most people outsi...,pos
17399,"'Apocalypse Now Redux', Francis Ford Coppola's...",pos
45725,The obvious parallels between Walt Disney's ch...,pos
40212,This was the Modesty that we didn't know! It w...,pos
629,"From hardly alien sounding lasers, to an eleme...",neg
...,...,...
34991,This film was one of the worst I've seen in a ...,neg
16805,I was a 20 year old college student living wit...,pos
13607,"When I first saw the movie, I thought it was s...",pos
37020,It's amazing that this movie turns out to be i...,neg


### Palabras de parada

In [3]:
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')
palabras_de_parada_ingles = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...'])
palabras_de_parada_ingles

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself'

### Normalizar el texto

In [4]:
# pip install pycontractions

In [5]:
import re

def limpiar_html(raw_html):
    reg_limpiar = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    texto_limpio = re.sub(reg_limpiar, '', raw_html)
    return texto_limpio

def normalizar_critica(critica):
    critica_sin_html = limpiar_html(critica)
    tokens = nltk.tokenize.casual.casual_tokenize(critica_sin_html, "english")
    tokens_normalizados = [token.lower() for token in tokens if token not in palabras_de_parada_ingles]
    return " ".join(tokens_normalizados)

def normalizar_fila(fila):
    nueva_fila = fila
    critica_normalizada = normalizar_critica(fila['Review'])
    nueva_fila['Review'] = critica_normalizada
    return nueva_fila

In [15]:
df_criticas_normalizadas = df_criticas.apply(normalizar_fila, axis=1)
df_criticas_normalizadas

,Review,Label
45748,it's really unfortunate people outside canada ...,pos
17399,apocalypse now redux francis ford coppola's wa...,pos
45725,the obvious parallels walt disney's charming f...,pos
40212,this modesty know it hinted summarized comic s...,pos
629,from hardly alien sounding lasers elementary s...,neg
...,...,...
34991,this film one worst i've seen long it's combin...,neg
16805,i 20 year old college student living folks i f...,pos
13607,when i first saw movie i thought sweet family ...,pos
37020,it's amazing movie turns one hitlists it far n...,neg


### Obtener el vocabulario del problema

In [7]:
def obtener_vocabulario_problema():
    cuerpo_texto_normalizado = ' '.join(df_criticas_normalizadas['Review'].tolist())
    vocabulario_problema = cuerpo_texto_normalizado.split()
    vocabulario_ordenado = sorted(set(vocabulario_problema))
    return vocabulario_ordenado

def obtener_vocabulario_problema_y_posicion(vocabulario_del_problema_ordenado):
    vocabulario_y_posicion = {}
    for i, token in enumerate(vocabulario_del_problema_ordenado):
        vocabulario_y_posicion[token] = i
    return vocabulario_y_posicion

In [16]:
vocabulario_problema = obtener_vocabulario_problema_y_posicion(obtener_vocabulario_problema())
#vocabulario_problema.keys()

### One Hot Encoding

In [17]:
import numpy as np

def obtener_one_hot_vector(critica, vocabulario_problema_y_posicion):
    one_hot_vector_vector = np.zeros(len(vocabulario_problema_y_posicion), dtype=int)
    for token in critica.split():
        one_hot_vector_vector[vocabulario_problema_y_posicion[token]] = 1
    return one_hot_vector_vector

In [18]:
one_hots = []
indices = []
for index, row in df_criticas_normalizadas.iterrows():
    one_hot = obtener_one_hot_vector(row['Review'], vocabulario_problema)
    indices.append(index)
    one_hots.append(one_hot)

## 2. Crear y entrenar el modelo predictivo

In [19]:
df_criticas_one_hot_vector = pd.DataFrame(columns=vocabulario_problema.keys(), data=one_hots, index=indices)
df_criticas_one_hot_vector['Target'] = df_criticas_normalizadas['Label']
X = df_criticas_one_hot_vector.drop('Target', axis=1).to_numpy()
Y = df_criticas_one_hot_vector['Target'].ravel()

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [21]:
from sklearn.linear_model import LogisticRegression
clasificador_reg_log = LogisticRegression(random_state=0, solver='liblinear')
clasificador_reg_log.fit(X_train, Y_train)
print('accuracy del clasificador: {0:.2f}'.format(clasificador_reg_log.score(X_test, Y_test)))

accuracy del clasificador: 0.85


### Guardar el clasificador como un archivo

In [24]:
import pickle
ruta_archivo_clasificador = os.path.join('clasificador-regresion-logistica.pkl')
archivo_clasificador = open(ruta_archivo_clasificador, 'wb')
pickle.dump(clasificador_reg_log, archivo_clasificador)
archivo_clasificador.close()

### Guardar el vocabulario del problema como un archivo

In [25]:
ruta_archivo_vocabulario = os.path.join('vocabulario-problema.pkl')
archivo_vacabulario = open(ruta_archivo_vocabulario, 'wb')
pickle.dump(vocabulario_problema, archivo_vacabulario)
archivo_vacabulario.close()

### Guardar las palabras de parada como un archivo

In [26]:
ruta_archivo_palabras_parada = os.path.join('palabras-parada.pkl')
archivo_palabras_parada = open(ruta_archivo_palabras_parada, 'wb')
pickle.dump(palabras_de_parada_ingles, archivo_palabras_parada)
archivo_palabras_parada.close()